# 네이버 뉴스 스크래핑 실습 (2020. 03. 12)

웹페이지를 스크래핑하는 원리는 본질적으로 크게 두 부분으로 나뉩니다. 첫째, URL주소에 접근해 해당 웹페이지에 대한 접근 권한을 얻습니다. 둘째, 해당 웹페이지를 구성하는 HTML구조 안의 태그 및 속성 정보들을 이용해 원하는 정보만을 취합니다. 첫 번째인 웹페이지에 대한 접근 권한은 주로 urlopen()이나 requests() 함수를 이용해 얻습니다. 두 번째인 해당 웹페이지 상에서 필요한 정보만을 취득하는 일은 BeautifulSoup()함수를 이용해 진행합니다.
  
실습 차원에서 '코로나'라는 검색어를 입력했을 때 나오는 네이버 뉴스 기사를 스크랩해야할 필요가 있다고 가정하겠습니다. 이 또한 크게 두 부분으로 나뉘어 진행됩니다. 먼저, 우리는 네이버 뉴스에 접속해 '코로나'라고 검색한 후 나타나는 기사 리스트에서, 해당 네이버 뉴스 기사로 이동할 수 있는 URL주소를 모두 스크랩합니다. 그리고 나서, 스크랩한 URL주소 각각에 접근해 스크랩을 진행합니다.

In [10]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import time

URL주소는 일정한 구조를 지니고 있습니다. 가령 네이버 뉴스에서 '코로나'라는 검색어를 입력한 후, 최신순으로 정렬했을 때 확인할 수 있는 URL은 아래와 같습니다.  

https://search.naver.com/search.naver?&where=news&query=%EC%BD%94%EB%A1%9C%EB%82%98&sm=tab_pge&sort=1&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:dd,p:all,a:all&mynews=0&start=1&refresh_start=0  

이중 base 주소는 https://search.naver.com/search.naver 이고 그 외는 구체적인 속성값을 의미합니다. 이 중 &start=1에서 다음페이지로 넘어가면 숫자가 11이되고 그 다음은 21이 되는식으로 변화해가는 패턴을 확인할 수 있습니다. 이에 착안해 변화하는 숫자 이전까지의 주소와 이후의 주소를 문자열로 저장하고 숫자만을 for문 안에서 변화시켜가며 각 기사의 네이버 뉴스 URL을 스크랩할 수 있습니다. 먼저 URL의 앞부분은 URL_HEAD라는 이름으로, 뒷 부분은 URL_TAIL이라는 이름으로 저장해두겠습니다.

In [1]:
URL_HEAD = 'https://search.naver.com/search.naver?&where=news&query=%EC%BD%94%EB%A1%9C%EB%82%98&sm=tab_pge&sort=1&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:dd,p:all,a:all&mynews=0&start='
URL_TAIL = '&refresh_start=0'

각각 10개의 기사를 열거하는 총 10개의 페이지 주소를 URL주소의 숫자 부분만 바꾸어 접근하기 위해 아래와 같이 for문을 구성할 수 있습니다.

In [3]:
for i in range(0, 10):
    url = URL_HEAD + str(i*10 + 1) + URL_TAIL
    print(url)

https://search.naver.com/search.naver?&where=news&query=%EC%BD%94%EB%A1%9C%EB%82%98&sm=tab_pge&sort=1&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:dd,p:all,a:all&mynews=0&start=1&refresh_start=0
https://search.naver.com/search.naver?&where=news&query=%EC%BD%94%EB%A1%9C%EB%82%98&sm=tab_pge&sort=1&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:dd,p:all,a:all&mynews=0&start=11&refresh_start=0
https://search.naver.com/search.naver?&where=news&query=%EC%BD%94%EB%A1%9C%EB%82%98&sm=tab_pge&sort=1&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:dd,p:all,a:all&mynews=0&start=21&refresh_start=0
https://search.naver.com/search.naver?&where=news&query=%EC%BD%94%EB%A1%9C%EB%82%98&sm=tab_pge&sort=1&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:dd,p:all,a:all&mynews=0&start=31&refresh_start=0
https://search.naver.com/search.naver?&where=news&query=%EC%BD%94%EB%A1%9C%EB%82%98&sm=tab_pge&sort=1&photo=0&field=0&reporter_article=&pd=0&ds=&de=&doci

이제 위에서 구성한 반복문을 진행하면서 총 10개의 페이지 내에서 네이버 뉴스에서 직접 제공하는 뉴스 기사의 링크에 모두 접근합니다.

In [9]:
for i in range(0, 10):
    url = URL_HEAD + str(i*10 + 1) + URL_TAIL
    html = urlopen(url)  # 첫번째 페이지의 URL주소에 대한 접근 권한을 얻고,
    soup = BeautifulSoup(html, 'lxml')  # 해당 HTML에서 파싱하기 위해 BeautifulSoup을 이용합니다.
    
    # 첫번째 페이지 내에 각 뉴스 기사와 관련된 태그 정보들을 담고 있는 태그에 접근합니다.
    article_list = soup.find('ul', {'class':'type01'}).find_all('li')
    
    # 10개의 원소로 구성된 리스트를 for문을 이용해 각각 접근한 후, 각각의 뉴스 기사에 해당하는 URL주소에 접근합니다.
    for article in article_list:
        url_sub = article.find('dd').find('a').get('href')
        # 만일 네이버 뉴스에서 직접 서비스하지 않는 경우 '#'이라고 나오기 때문에 이러한 경우는 조건문으로 배제합니다.
        if url_sub == '#':  
            continue
        print(url_sub)  # 네이버 뉴스에서 직접 서비스하는 뉴스 각각의 URL주소
    print(i)  # 페이지 번호

https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&oid=410&aid=0000676136
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=437&aid=0000233488
0
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=022&aid=0003446630
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=055&aid=0000799912
1
http://sports.news.naver.com/kbaseball/news/read.nhn?oid=119&aid=0002388746
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=055&aid=0000799912
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=437&aid=0000233486
2
http://sports.news.naver.com/kbaseball/news/read.nhn?oid=119&aid=0002388746
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=055&aid=0000799912
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=448&aid=0000293775
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0011469405
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=437&aid=00

아래와 같이 코드를 수정하면, 위에서 접근한 네이버 뉴스 URL 주소를 빈 리스트를 만들어 담아둡니다.

In [21]:
url_list = []

for i in range(0, 10):
    url = URL_HEAD + str(i*10 + 1) + URL_TAIL
    html = urlopen(url)
    soup = BeautifulSoup(html, 'lxml') 
    article_list = soup.find('ul', {'class':'type01'}).find_all('li')
    for article in article_list:
        url = article.find('dd').find('a').get('href')
        if url == '#':
            continue
        url_list.append(url)

In [23]:
len(url_list)  # 몇 개의 URL이 수집되었는지 확인

23

In [24]:
url_list

['http://sports.news.naver.com/basketball/news/read.nhn?oid=351&aid=0000046516',
 'http://sports.news.naver.com/basketball/news/read.nhn?oid=351&aid=0000046516',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=011&aid=0003708943',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=081&aid=0003073303',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=421&aid=0004519745',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=028&aid=0002489129',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=028&aid=0002489129',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=015&aid=0004305655',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=056&aid=0010802925',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=312&aid=0000438991',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=056&aid=0010802924',
 'https://news.naver.com/mai

이제 위에서 수집한 각각의 뉴스 URL에 접근해서 뉴스의 제목과 작성시간, 그리고 언론사 이름을 스크랩해서 txt파일로 저장해둡니다. 그 과정에서 네이버 스포츠 뉴스 기사의 경우 일반 뉴스 기사와 HTML의 태그 구조가 다르기 때문에 조건문(if 문)을 활용해서 다르게 적용시켜줍니다. 더불어, 혹시모를 에러의 발생 상황을 대비해서 try와 except를 이용해 예외조치를 해둡니다.

In [32]:
with open('url_list.txt', 'w', encoding='utf-8') as f:  
    for url in url_list:
        html = urlopen(url)
        soup = BeautifulSoup(html, 'lxml')
        try:
            if soup.find('h3', id='articleTitle') == None:
                title = soup.find('h4', class_='title').text
                press = soup.find('p', class_="source").text.split(' ')[1]
                date = soup.find('div', class_="info").text.split(' ')[1]
                f.write(title + '\t' + press + '\t' + date + '\n')
            else:
                title = soup.find('h3', id='articleTitle').text
                press = soup.find('meta', property="me2:category1").get('content')
                date = soup.find('span', class_='t11').text.split(' ')[0]
                f.write(title + '\t' + press + '\t' + date + '\n')
        except:
            print("error")

error
error
